In [ ]:
from anonymizer.detection import Detector, get_weights_path
from anonymizer.obfuscation import Obfuscator
from PIL import Image
import numpy as np
import math
import requests


In [ ]:
weights_path = './weights'
kernel_size, sigma, box_kernel_size = 21,2,9
obfuscator = Obfuscator(kernel_size=int(kernel_size), sigma=float(sigma), box_kernel_size=int(box_kernel_size))
detectors = {
    'face': Detector(kind='face', weights_path=get_weights_path(weights_path, kind='face')),
    'plate': Detector(kind='plate', weights_path=get_weights_path(weights_path, kind='plate'))
}
detection_thresholds = {
    'face': 0.3,
    'plate': 0.3
}

In [ ]:
image = Image.open(requests.get("https://raw.githubusercontent.com/understand-ai/anonymizer/master/images/coco01.jpg", stream=True).raw).convert('RGB')

In [ ]:
def start_points(size, split_size, overlap=0, region_of_interest=None):
    offset = 0

    if region_of_interest:
        size = region_of_interest[1] - region_of_interest[0]
        offset = region_of_interest[0]
    
    stride = int(split_size * (1-overlap))
    
    points = []
    counter = 0
    
    while True:
        start_point = stride * counter
        
        if start_point + split_size >= size: # last box, will potentially exceed size
            points.append((start_point+offset, size+offset))
            break
        
        points.append((start_point+offset, start_point+split_size+offset))
        counter += 1
    return points

In [ ]:
split_width = 2000
split_height = 2500
overlap = 0.1 # 10% overlap for both x and y

# for equirrectangular panoramas, we can ignore the upper and lower thirds
if math.floor(image.width / image.height) == 2:
    roi_start = math.floor(image.height * 0.4)
    roi_stop = math.floor(image.height * 0.75)
    region_of_interest = [roi_start, roi_stop]
else:
    region_of_interest = None

X_points = start_points(image.width, split_width, overlap=overlap)
Y_points = start_points(image.height, split_height, overlap=overlap, region_of_interest=region_of_interest)

X_points, Y_points

In [ ]:
coords = []
for x_start, x_stop in X_points:
    for y_start, y_stop in Y_points:
        coords.append((x_start, x_stop, y_start, y_stop))

print(f'Total of {len(coords)} splits')
coords

In [ ]:
detections = []

from anonymizer.utils import Box

for x_start, x_stop, y_start, y_stop in coords:
    tile = image.crop((x_start, y_start, x_stop, y_stop))
    np_tile = np.array(tile)

    for kind, detector in detectors.items():
        boxes = detector.detect(np_tile, detection_threshold=detection_thresholds[kind])

        for detection in boxes:
            detections.append(Box(
                detection.x_min + x_start,
                detection.y_min + y_start,
                detection.x_max + x_start,
                detection.y_max + y_start,
                detection.score,
                detection.kind
            ))

detections

In [ ]:
np_image = np.array(image)
obfuscated_np_image = obfuscator.obfuscate(np_image, detections)

In [ ]:
obfuscated_image = Image.fromarray((obfuscated_np_image).astype(np.uint8), mode='RGB')
obfuscated_image.save(f"/Users/george/Sites/anonymizer/images-anonymized/result.jpg")